# Notebook Summary

This notebook was used to generate augmented waveforms which were then transformed into mel spectrograms and fed into CNN3 in `SoundIdentification_Final (V2).ipynb`.

_____________
_____________

# Importing Libraries and code from **main.py**

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# Importing Libraries
import librosa
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import random
import pandas as pd
import os
from tqdm import tqdm
from sklearn.preprocessing import normalize

In [ ]:
meta_file_path = '/content/drive/My Drive/UCL/ML_Y3/Project/meta/sounds.csv'
signal_file_path = '/content/drive/My Drive/UCL/ML_Y3/Project/'


def getMeta():
    return pd.read_csv('/content/drive/My Drive/UCL/ML_Y3/Project/meta/sounds.csv')

def getSignal(fileName):
    # Construct the full file path
    file_path = signal_file_path + fileName
    # Load the audio file
    return librosa.load(file_path, sr=44100)[0]

def getSpect(signal):
    spec = librosa.feature.melspectrogram(y=signal, sr=44100, n_fft=2205, hop_length=441)
    spec = librosa.power_to_db(spec)
    return spec

def getSignalSpect(meta,number):
    recording=meta['filename'][number]
    signal = getSignal('audio/'+ recording)
    return signal, getSpect(signal)

# My function
def getSignalSpectSynth(meta, number, ver=1):
    """
    Retrieve the signal and its corresponding spectrogram from a specified version directory.

    Parameters:
        meta (Series DataFrame): audio metadata --> filenames.
        number (int): The index number to access the filename from the 'meta' DataFrame.
        ver (int, optional): The version of the audio dataset directory to use. Defaults to 1.
        raises an error unless ver = 1,2 or 3


    Returns:
        tuple: A tuple containing the audio signal and its spectrogram.

    """
    recording=meta['filename'][number]
    if ver == 1:
      signal = getSignal('audio/synthetic_audio/'+ recording)
    elif ver == 2:
      signal = getSignal('audio/synthetic_audio_V2/'+ recording)
    elif ver == 3:
      signal = getSignal('audio/synthetic_audio_V3/'+ recording)
    else:
        raise ValueError(f"Unsupported version: {ver}")
    return signal, getSpect(signal)


meta_df=getMeta()

# Generating Synthetic Audio Data - Task 1

At this point I realize that maybe we can use synthetic audio data to increase the training set with data that has more variance which should allow for higher accuracy, as more variance can be captured for specific labels.

## Method 1

Defining functions used to apply augmentations

In [ ]:
def random_pitch_shift(y, sr, min_steps=-4, max_steps=4):
    """
    Randomly shifts the pitch of the audio.
    """
    n_steps = random.uniform(min_steps, max_steps)
    return librosa.effects.pitch_shift(y, sr=sr, n_steps=n_steps)

def random_time_stretch(y, sr, min_rate=0.8, max_rate=1.2):
    """
    Randomly stretches the time of the audio.
    """
    rate = random.uniform(min_rate, max_rate)
    return librosa.effects.time_stretch(y, rate=rate)

def apply_random_augmentation(audio_path, output_path, sr=44100):
    """
    Applies a random augmentation to the audio file.
    """
    y, sr = librosa.load(audio_path)

    # Choose a random augmentation
    augmentation_choice = random.choice([random_pitch_shift, random_time_stretch])

    # Apply the augmentation
    y_augmented = augmentation_choice(y, sr)

    # Save the augmented audio
    sf.write(output_path, y_augmented, sr)

    return y, sr

_________________
Saving audio files on drive. **DO NOT RE-RUN**

In [ ]:
# Path setup
audio_files_directory = '/content/drive/My Drive/UCL/ML_Y3/Project/audio'
output_directory = '/content/drive/My Drive/UCL/ML_Y3/Project/audio/synthetic audio'
audio_files = os.listdir(audio_files_directory)

meta = getMeta()

for index, row in meta[['filename', 'target']].iterrows():
    filename = row['filename']
    label = row['target']
    # Identifying specific audio file wihch will be augmented
    audio_path = os.path.join(audio_files_directory, filename)

    # Generate a unique output filename including the original filename
    output_filename = f"aug_{filename}"
    output_path = os.path.join(output_directory, output_filename)

    # Calling the pre-defined function and saving the audio files
    signal, sr = apply_random_augmentation(audio_path, output_path)

    print(f"{index}: Processed and saved: {output_filename} with label: {label}")

In [ ]:
# Using the data frame below to call the audio files from drive
# and turn them into arrays
meta_synth = getMeta()
meta_synth['filename'] = "aug_" + meta_synth['filename']

,filename,fold,target,category,esc10,src_file,take
0,aug_1-100032-A-0.wav,1,0,dog,True,100032,A
1,aug_1-100038-A-14.wav,1,14,chirping_birds,False,100038,A
2,aug_1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A
3,aug_1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B
4,aug_1-101296-A-19.wav,1,19,thunderstorm,False,101296,A
...,...,...,...,...,...,...,...
1995,aug_5-263831-B-6.wav,5,6,hen,False,263831,B
1996,aug_5-263902-A-36.wav,5,36,vacuum_cleaner,False,263902,A
1997,aug_5-51149-A-25.wav,5,25,footsteps,False,51149,A
1998,aug_5-61635-A-8.wav,5,8,sheep,False,61635,A


### Data Synthesis Problem
We run into a problem. When augmenting the time sometimes we reduce or increase the x-axis (time axis) of the generated spectogram. So, before generating the spectrogram images we need a function that will normalize the time to 500, by padding or trimming the arrays.

In [ ]:
def fix_audio_length(y, sr, fixed_length=500):
    """
    Adjusts the audio signal to a fixed length by either padding with zeros or trimming.
    """
    # Calculate the fixed number of samples
    #fixed_samples = sr * fixed_length
    fixed_samples = fixed_length
    if len(y) < fixed_samples:
        # If the audio is shorter than the desired fixed length, pad it
        y_padded = librosa.util.fix_length(y, size = fixed_samples)
        return y_padded
    else:
        # If the audio is longer, trim it
        return y[:fixed_samples]

**DO NOT RERUN**. Below we save the augmented data in google drive for further use.

In [ ]:
data_synth = np.zeros((2000,128,500))

# sr is the same for the augmented images
sr = 44100

for i in tqdm(range(0, 2000)):
    spec = normalize(getSignalSpectSynth(meta_synth, i)[1])
    spec = fix_audio_length(spec, sr) # Keeping the audio length 500
    print(spec.shape)
    data_synth[i] = spec
    print(data_synth.shape)

# Saving it on google drive so we don't have to re-run the code
save_synth_file_path = '/content/drive/My Drive/UCL/ML_Y3/Project/images/synthetic_spectogram_data_array.npy'
np.save(save_synth_file_path, data_synth)

#### Initial saving method (Incorrect)



```
# Path setup
audio_files_directory = '/content/drive/My Drive/UCL/ML_Y3/Project/audio'
output_directory = '/content/drive/My Drive/UCL/ML_Y3/Project/audio/synthetic audio'
audio_files = os.listdir(audio_files_directory)

meta = getMeta()

for index, row in meta[['filename', 'target']].iterrows():
    filename = row['filename']
    label = row['target']
    audio_path = os.path.join(audio_files_directory, filename)
    # Generating a unique output filename that includes the original filename
    # This helps maintain the mapping to the original label
    output_filename = f"aug_{filename}"  # Customize this as needed
    output_path = os.path.join(output_directory, output_filename)

    signal, sr = apply_random_augmentation(audio_path, output_path)
    # No need to manually map the augmented file to its label;
    # The filename contains the necessary info to trace back to the original label

    print(f"{index}: Processed and saved: {output_filename} with label: {label}")
```



## Method 2

In method 1 we apply either pitch shift or time shift to our data set. This means that we double it in size. It definetly improves the performance of our models, however, what if we made sure to apply both pitch shift and time stretch (maybe add noise too). This would triple our dataset.

In [ ]:
def apply_stretch_shift(audio_path, output_path, filename, sr=44100):
    """
    Applies a random augmentation to the audio file.
    """
    signal, sr = librosa.load(audio_path)

    # Apply the augmentations
    sig_pitch_shift = random_pitch_shift(signal, sr)
    sig_time_stretch = random_time_stretch(signal, sr)
    #sig_white_noise = add_white_noise(signal, sr)
    #sig_pink_noise = add_pink_noise(signal, sr)


    # Setting up the the output paths for the .wav files

    # Shift augmentation
    shift_filename = f"shift_{filename}"
    shift_path = os.path.join(output_directory, shift_filename)

    # Strech augmentation
    stretch_filename = f"stretch_{filename}"
    stretch_path = os.path.join(output_directory, stretch_filename)

    # Save the augmented audio
    sf.write(shift_path, sig_pitch_shift, sr)
    sf.write(stretch_path, sig_time_stretch, sr)
    #sf.write(output_path, sig_white_noise, sr)
    #sf.write(output_path, sig_pink_noise, sr)

    return signal, sr, shift_filename, stretch_filename

### Saving Files

#### Saving audio files .wav

In [ ]:
audio_files_directory = '/content/drive/My Drive/UCL/ML_Y3/Project/audio'
output_directory = '/content/drive/My Drive/UCL/ML_Y3/Project/audio/synthetic_audio_V2'
audio_files = os.listdir(audio_files_directory)

meta = getMeta()

for index, row in tqdm(meta[['filename', 'target']].iterrows(), total=len(meta)): # tqdm was not necessary...
    filename = row['filename']
    label = row['target']
    # Identifying a specific sound file which will be augmented
    audio_path = os.path.join(audio_files_directory, filename)

    # Applying the augmentaitons
    signal, sr, shift_filename, stretch_filename = apply_stretch_shift(audio_path, output_directory, filename)

    if index % 10 == 9:
      print(f"\n{index}: Processed and saved: {shift_filename} with label: {label}")
      print(f"{index}: Processed and saved: {stretch_filename} with label: {label}")


  0%|          | 0/400.0 [00:00<?, ?it/s]
  0%|          | 1/400.0 [00:00<00:54,  7.34it/s]
  0%|          | 2/400.0 [00:00<01:03,  6.25it/s]
  1%|          | 3/400.0 [00:00<01:06,  5.99it/s]
  1%|          | 4/400.0 [00:00<01:04,  6.13it/s]
  1%|▏         | 5/400.0 [00:00<01:07,  5.89it/s]
  2%|▏         | 6/400.0 [00:01<01:15,  5.20it/s]
  2%|▏         | 7/400.0 [00:01<01:19,  4.96it/s]
  2%|▏         | 8/400.0 [00:01<01:23,  4.69it/s]
  2%|▏         | 9/400.0 [00:01<01:27,  4.48it/s]
  2%|▎         | 10/400.0 [00:01<01:25,  4.58it/s]


9: Processed and saved: shift_1-103995-A-30.wav with label: 30
9: Processed and saved: stretch_1-103995-A-30.wav with label: 30



  3%|▎         | 11/400.0 [00:02<01:23,  4.68it/s]
  3%|▎         | 12/400.0 [00:02<01:21,  4.74it/s]
  3%|▎         | 13/400.0 [00:02<01:25,  4.53it/s]
  4%|▎         | 14/400.0 [00:02<01:26,  4.45it/s]
  4%|▍         | 15/400.0 [00:03<01:21,  4.71it/s]
  4%|▍         | 16/400.0 [00:03<01:26,  4.46it/s]
  4%|▍         | 17/400.0 [00:03<01:27,  4.36it/s]
  4%|▍         | 18/400.0 [00:03<01:30,  4.22it/s]
  5%|▍         | 19/400.0 [00:04<01:30,  4.19it/s]
  5%|▌         | 20/400.0 [00:04<01:33,  4.07it/s]


19: Processed and saved: shift_1-115545-C-48.wav with label: 48
19: Processed and saved: stretch_1-115545-C-48.wav with label: 48



  5%|▌         | 21/400.0 [00:04<01:34,  4.03it/s]
  6%|▌         | 22/400.0 [00:04<01:33,  4.05it/s]
  6%|▌         | 23/400.0 [00:05<01:29,  4.20it/s]
  6%|▌         | 24/400.0 [00:05<01:22,  4.53it/s]
  6%|▋         | 25/400.0 [00:05<01:19,  4.74it/s]
  6%|▋         | 26/400.0 [00:05<01:13,  5.07it/s]
  7%|▋         | 27/400.0 [00:05<01:11,  5.19it/s]
  7%|▋         | 28/400.0 [00:05<01:07,  5.49it/s]
  7%|▋         | 29/400.0 [00:06<01:04,  5.72it/s]
  8%|▊         | 30/400.0 [00:06<01:06,  5.52it/s]
  8%|▊         | 31/400.0 [00:06<01:03,  5.77it/s]

...


1979: Processed and saved: shift_5-260432-A-39.wav with label: 39
1979: Processed and saved: stretch_5-260432-A-39.wav with label: 39



1982it [06:38,  5.92it/s]
1983it [06:39,  5.79it/s]
1984it [06:39,  5.91it/s]
1985it [06:39,  6.05it/s]
1986it [06:39,  5.45it/s]
1987it [06:39,  4.61it/s]
1988it [06:40,  4.32it/s]
1989it [06:40,  4.28it/s]
1990it [06:40,  4.13it/s]


1989: Processed and saved: shift_5-263490-A-25.wav with label: 25
1989: Processed and saved: stretch_5-263490-A-25.wav with label: 25



1991it [06:40,  4.13it/s]
1992it [06:41,  4.14it/s]
1993it [06:41,  4.32it/s]
1994it [06:41,  4.19it/s]
1995it [06:41,  4.15it/s]
1996it [06:42,  4.12it/s]
1997it [06:42,  4.37it/s]
1998it [06:42,  4.56it/s]
1999it [06:42,  4.46it/s]
2000it [06:42,  4.96it/s]


1999: Processed and saved: shift_5-9032-A-0.wav with label: 0
1999: Processed and saved: stretch_5-9032-A-0.wav with label: 0


In [ ]:
audio_files_directory = '/content/drive/My Drive/UCL/ML_Y3/Project/audio'
output_directory = '/content/drive/My Drive/UCL/ML_Y3/Project/audio/synthetic_audio_V2'

# Checking the number of files that were saved in the output directory
num_files = len([f for f in os.listdir(output_directory) if os.path.isfile(os.path.join(output_directory, f))])
print(f'There were {num_files} number of .wav files saved in the folder')

There were 3990 number of .wav files saved in the folder


Should be 4000. So, some of the files were not saved onto the file path. We will find the missing files and not use them for training...

##### Debugging

In [ ]:
original_files = os.listdir(audio_files_directory)

expected_files = set()
for filename in original_files:
    expected_files.add(f"stretch_{filename}")
    expected_files.add(f"shift_{filename}")

# Create a set of actual generated file names in the output directory
actual_files = set(os.listdir(output_directory))

# Find the difference - files that were expected but not found
missing_files = expected_files - actual_files

# Removing folders that were displayed in the directory
missing_files = missing_files - set(['shift_synthetic_audio','shift_synthetic_audio_V2',
                                     'stretch_synthetic_audio_V2', 'stretch_synthetic_audio_V2',
                                     'stretch_synthetic_audio', 'shift_synthetic_audio_V3',
                                     'stretch_synthetic_audio_V3'])


missing_shift_files = {file for file in missing_files if file.startswith('shift_')}
missing_stretch_files = {file for file in missing_files if file.startswith('stretch_')}

print(f"Missing pitch shifted files ({len(missing_shift_files)}):")
print(missing_shift_files)
print(f"\nMissing time stretched files ({len(missing_shift_files)}):")
print(f'{missing_stretch_files}')

Missing pitch shifted files (5):
{'shift_1-115545-B-48.wav', 'shift_1-115545-C-48.wav', 'shift_1-115546-A-48.wav', 'shift_1-115521-A-19.wav', 'shift_1-115545-A-48.wav'}

Missing time stretched files (5):
{'stretch_1-115521-A-19.wav', 'stretch_1-115545-B-48.wav', 'stretch_1-115545-C-48.wav', 'stretch_1-110537-A-22.wav', 'stretch_1-115545-A-48.wav'}


### Saving Files

#### Creating pandas dfs to store .wav files

Creating a df to store the correct filename metadata for pitch shifted .wav files, by altering the filenames

In [ ]:
# Generating a df for pitch shifted data
meta_shift = getMeta()
meta_shift['filename'] = "shift_" + meta_shift['filename']

# Checking which files are missin (4 fireworks and 1 thunderstorm)
meta_shift[meta_shift['filename'].isin(missing_shift_files)] # Displaying df

,filename,fold,target,category,esc10,src_file,take
16,shift_1-115521-A-19.wav,1,19,thunderstorm,False,115521,A
17,shift_1-115545-A-48.wav,1,48,fireworks,False,115545,A
18,shift_1-115545-B-48.wav,1,48,fireworks,False,115545,B
19,shift_1-115545-C-48.wav,1,48,fireworks,False,115545,C
20,shift_1-115546-A-48.wav,1,48,fireworks,False,115546,A


In [ ]:
# Pruning the data frame
meta_shift = meta_shift[~meta_shift['filename'].isin(missing_shift_files)]
meta_shift.reset_index(drop=True, inplace=True) # Reseting the indeces
meta_shift

,filename,fold,target,category,esc10,src_file,take
0,shift_1-100032-A-0.wav,1,0,dog,True,100032,A
1,shift_1-100038-A-14.wav,1,14,chirping_birds,False,100038,A
2,shift_1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A
3,shift_1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B
4,shift_1-101296-A-19.wav,1,19,thunderstorm,False,101296,A
...,...,...,...,...,...,...,...
1990,shift_5-263831-B-6.wav,5,6,hen,False,263831,B
1991,shift_5-263902-A-36.wav,5,36,vacuum_cleaner,False,263902,A
1992,shift_5-51149-A-25.wav,5,25,footsteps,False,51149,A
1993,shift_5-61635-A-8.wav,5,8,sheep,False,61635,A


In [ ]:
# For time stretched data
meta_stretch = getMeta()
meta_stretch['filename'] = "stretch_" + meta_stretch['filename']

# Missing 3 fireworks, 1 thunderstorm and 1 clapping
meta_stretch[meta_stretch['filename'].isin(missing_stretch_files)]

,filename,fold,target,category,esc10,src_file,take
15,stretch_1-110537-A-22.wav,1,22,clapping,False,110537,A
16,stretch_1-115521-A-19.wav,1,19,thunderstorm,False,115521,A
17,stretch_1-115545-A-48.wav,1,48,fireworks,False,115545,A
18,stretch_1-115545-B-48.wav,1,48,fireworks,False,115545,B
19,stretch_1-115545-C-48.wav,1,48,fireworks,False,115545,C


In [ ]:
# Pruning the dataframe
meta_stretch = meta_stretch[~meta_stretch['filename'].isin(missing_stretch_files)]
meta_stretch.reset_index(drop=True, inplace=True) # Reseting the indeces
meta_stretch

,filename,fold,target,category,esc10,src_file,take
0,stretch_1-100032-A-0.wav,1,0,dog,True,100032,A
1,stretch_1-100038-A-14.wav,1,14,chirping_birds,False,100038,A
2,stretch_1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A
3,stretch_1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B
4,stretch_1-101296-A-19.wav,1,19,thunderstorm,False,101296,A
...,...,...,...,...,...,...,...
1990,stretch_5-263831-B-6.wav,5,6,hen,False,263831,B
1991,stretch_5-263902-A-36.wav,5,36,vacuum_cleaner,False,263902,A
1992,stretch_5-51149-A-25.wav,5,25,footsteps,False,51149,A
1993,stretch_5-61635-A-8.wav,5,8,sheep,False,61635,A


#### Saving as .npy arrays in Google Drive

Before pruning we were getting errors here. **DO NOT RERUN** if you rerun, delete filies in the directory.

##### Saving Pitch Shifted files

In [ ]:
data_shift = np.zeros((1995,128,500)) # we have 1995 shifted spectrograms

# sr is the same for the augmented images
sr = 44100

for i in tqdm(range(0, 1995)):
    spec = normalize(getSignalSpectSynth(meta_shift, i, ver = 2)[1]) # Looping through only the shifted files
    spec = fix_audio_length(spec, sr) # Keeping the audio length 500
    data_shift[i] = spec


# Saving it on google drive so we don't have to re-run the code
save_shift_file_path = '/content/drive/My Drive/UCL/ML_Y3/Project/images/Data_Arrays/shifted_spectogram_data_array.npy'
np.save(save_shift_file_path, data_shift)

100%|██████████| 1995/1995 [07:35<00:00,  4.38it/s]


##### Saving Pitch Stretched files

In [ ]:
data_stretch = np.zeros((1995,128,500))

# sr is the same for the augmented images
sr = 44100

for i in tqdm(range(0, 1995)):
    spec = normalize(getSignalSpectSynth(meta_stretch, i, ver = 2)[1])
    spec = fix_audio_length(spec, sr) # Keeping the audio length 500
    data_stretch[i] = spec


# Saving it on google drive so we don't have to re-run the code
save_stretch_file_path = '/content/drive/My Drive/UCL/ML_Y3/Project/images/Data_Arrays/stretched_spectogram_data_array.npy'
np.save(save_stretch_file_path, data_shift)

100%|██████████| 1995/1995 [11:52<00:00,  2.80it/s]


#### Debugging again
**When re-running the data generation code after several days, it seems that i am now missing different files..**
The method of debuggin i.e pruning away the augmented data was not effective because we ran into a seperate problem. The size mismatch between the augmented data and the original dataset does not allow us to split the data sets in the same way.This means that when we are training the models we are using the augmented version of the test data in our training data, superflously overexagerating our model accuracy...

In [ ]:
audio_files_directory = '/content/drive/My Drive/UCL/ML_Y3/Project/audio'
output_directory = '/content/drive/My Drive/UCL/ML_Y3/Project/audio/synthetic_audio_V2/missing files'
audio_files = os.listdir(audio_files_directory)

meta = getMeta()

# It was noticed that only indices from 14 to 21 were missing
# for both the stretched and the shifted datasets.
# So we will only generate these specific sounds.

for index, row in meta[['filename', 'target']].iterrows(): # tqdm was not necessary...
    # Identifying a specific sound file which will be augmented
    if 20 <= index <= 36:
        filename = row['filename']
        label = row['target']
        audio_path = os.path.join(audio_files_directory, filename)

        # Applying the augmentations
        signal, sr, shift_filename, stretch_filename = apply_stretch_shift(audio_path, output_directory, filename)
    else:
        # If the index is greater than 21, break the loop
        if index > 36:
            break
        # Continue to the next iteration for indices less than 15
        continue

  # The relevant ones were chosen
  # And moved back to the main directory in order to extract .npy files


In [ ]:
# we will save the files again as before but add back the 5 missing
# data points to each of the .npy files
meta_shift = getMeta()
meta_shift['filename'] = "shift_" + meta_shift['filename']

data_shift = np.zeros((2000,128,500)) # we have 2000 shifted spectrograms

# sr is the same for the augmented images
sr = 44100

for i in tqdm(range(0, 2000)):
    spec = normalize(getSignalSpectSynth(meta_shift, i, ver = 2)[1]) # Looping through only the shifted files
    spec = fix_audio_length(spec, sr) # Keeping the audio length 500
    data_shift[i] = spec


# Saving it on google drive so we don't have to re-run the code
save_shift_file_path = '/content/drive/My Drive/UCL/ML_Y3/Project/images/Data_Arrays/FULL_shifted_spectogram_data_array.npy'
np.save(save_shift_file_path, data_shift)

100%|██████████| 2000/2000 [11:02<00:00,  3.02it/s]


In [ ]:
audio_files_directory = '/content/drive/My Drive/UCL/ML_Y3/Project/audio'
output_directory = '/content/drive/My Drive/UCL/ML_Y3/Project/audio/synthetic_audio_V2'

# Checking the number of files that were saved in the output directory
num_files = len([f for f in os.listdir(output_directory) if os.path.isfile(os.path.join(output_directory, f))])
print(f'There were {num_files} number of .wav files saved in the folder')

original_files = os.listdir(audio_files_directory)

expected_files = set()
for filename in original_files:
    expected_files.add(f"stretch_{filename}")
    expected_files.add(f"shift_{filename}")

# Create a set of actual generated file names in the output directory
actual_files = set(os.listdir(output_directory))

# Find the difference - files that were expected but not found
missing_files = expected_files - actual_files

# Removing folders that were displayed in the directory
missing_files = missing_files - set(['shift_synthetic_audio','shift_synthetic_audio_V2',
                                     'stretch_synthetic_audio_V2', 'stretch_synthetic_audio_V2',
                                     'stretch_synthetic_audio', 'shift_synthetic_audio_V3',
                                     'stretch_synthetic_audio_V3'])


missing_shift_files = {file for file in missing_files if file.startswith('shift_')}
missing_stretch_files = {file for file in missing_files if file.startswith('stretch_')}

print(f"Missing pitch shifted files ({len(missing_shift_files)}):")
print(missing_shift_files)
print(f"\nMissing time stretched files ({len(missing_stretch_files)}):")
print(f'{missing_stretch_files}')

There were 3975 number of .wav files saved in the folder
Missing pitch shifted files (10):
{'shift_1-13571-A-46.wav', 'shift_1-137-A-32.wav', 'shift_1-13613-A-37.wav', 'shift_1-118559-A-17.wav', 'shift_1-121951-A-8.wav', 'shift_1-12654-A-15.wav', 'shift_1-12654-B-15.wav', 'shift_1-12653-A-15.wav', 'shift_1-119125-A-45.wav', 'shift_1-13572-A-46.wav'}

Missing time stretched files (16):
{'stretch_1-13572-A-46.wav', 'stretch_1-12654-B-15.wav', 'stretch_1-118206-A-31.wav', 'stretch_1-12654-A-15.wav', 'stretch_1-116765-A-41.wav', 'stretch_1-115921-A-22.wav', 'stretch_1-118559-A-17.wav', 'stretch_1-13613-A-37.wav', 'stretch_1-13571-A-46.wav', 'stretch_1-115546-A-48.wav', 'stretch_1-121951-A-8.wav', 'stretch_1-115920-B-22.wav', 'stretch_1-115920-A-22.wav', 'stretch_1-11687-A-47.wav', 'stretch_1-12653-A-15.wav', 'stretch_1-119125-A-45.wav'}


In [ ]:
# For time stretched data
meta_stretch = getMeta()
meta_stretch['filename'] = "stretch_" + meta_stretch['filename']

# Missing 3 fireworks, 1 thunderstorm and 1 clapping
meta_stretch[meta_stretch['filename'].isin(missing_stretch_files)]

,filename,fold,target,category,esc10,src_file,take
20,stretch_1-115546-A-48.wav,1,48,fireworks,False,115546,A
21,stretch_1-115920-A-22.wav,1,22,clapping,False,115920,A
22,stretch_1-115920-B-22.wav,1,22,clapping,False,115920,B
23,stretch_1-115921-A-22.wav,1,22,clapping,False,115921,A
24,stretch_1-116765-A-41.wav,1,41,chainsaw,True,116765,A
25,stretch_1-11687-A-47.wav,1,47,airplane,False,11687,A
26,stretch_1-118206-A-31.wav,1,31,mouse_click,False,118206,A
27,stretch_1-118559-A-17.wav,1,17,pouring_water,False,118559,A
28,stretch_1-119125-A-45.wav,1,45,train,False,119125,A
29,stretch_1-121951-A-8.wav,1,8,sheep,False,121951,A


In [ ]:
# Generating a df for pitch shifted data
meta_shift = getMeta()
meta_shift['filename'] = "shift_" + meta_shift['filename']

# Checking which files are missin (4 fireworks and 1 thunderstorm)
meta_shift[meta_shift['filename'].isin(missing_shift_files)] # Displaying df

,filename,fold,target,category,esc10,src_file,take
27,shift_1-118559-A-17.wav,1,17,pouring_water,False,118559,A
28,shift_1-119125-A-45.wav,1,45,train,False,119125,A
29,shift_1-121951-A-8.wav,1,8,sheep,False,121951,A
30,shift_1-12653-A-15.wav,1,15,water_drops,False,12653,A
31,shift_1-12654-A-15.wav,1,15,water_drops,False,12654,A
32,shift_1-12654-B-15.wav,1,15,water_drops,False,12654,B
33,shift_1-13571-A-46.wav,1,46,church_bells,False,13571,A
34,shift_1-13572-A-46.wav,1,46,church_bells,False,13572,A
35,shift_1-13613-A-37.wav,1,37,clock_alarm,False,13613,A
36,shift_1-137-A-32.wav,1,32,keyboard_typing,False,137,A


In [ ]:
# For time stretched data
meta_stretch = getMeta()
meta_stretch['filename'] = "stretch_" + meta_stretch['filename']


data_stretch = np.zeros((2000,128,500))

# sr is the same for the augmented images
sr = 44100

for i in tqdm(range(0, 2000)):
    spec = normalize(getSignalSpectSynth(meta_stretch, i, ver = 2)[1])
    spec = fix_audio_length(spec, sr) # Keeping the audio length 500
    data_stretch[i] = spec


# Saving it on google drive so we don't have to re-run the code
save_stretch_file_path = '/content/drive/My Drive/UCL/ML_Y3/Project/images/Data_Arrays/FULL_stretched_spectogram_data_array.npy'
np.save(save_stretch_file_path, data_shift)

100%|██████████| 2000/2000 [01:57<00:00, 17.07it/s]


## Method 3

Now we will try to add augmentations which add white and pink noise onto our dataset.

We chose the SNR ratios by examining papers.

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5824438/

### Defining Functions

1. White
2. Pink noises
3. To adjust their SNR

So that the noise applied is roughly of the same level depending on the signal.

In [ ]:
# We considered adding brown noise, however it has features very simillar to
# pink noise (it just falls of faster).

def adjust_SNR(signal, noise, snr=[10,17]):
    '''
    Parameters:
        signal (ndarray): The input signal to which noise is added.
        noise (ndarray): The noise to be added to the signal
        snr (float): range of Possible signal to noise ratios

    Returns:
        ndarray: The signal with pink noise added.
    '''
    snr = np.random.uniform(*snr)  # Use asterik to unpack the values into single elements

    # Ensuring SNR does not lead to overflow
    snr_val = 10**(snr / 20)
    if np.isinf(snr_val) or np.isnan(snr_val):
        # Handle extreme SNR values or set to a default
        snr_val = 10**(10.5 / 20)  # Example: default to 7 dB

    # Calculate rms and rms noise
    rms_signal = np.sqrt(np.mean(signal**2))
    rms_noise = rms_signal / (10**(snr_val / 20))

    noise *= rms_noise / np.sqrt(np.mean(noise**2))
    return noise

                        ##############

def add_pink_noise(signal): #snr=[10,17]
    """
    Add pink noise to the input signal.

    Parameters:
        signal (ndarray): The input signal to which pink noise will be added.
        snr (float): The desired Signal-to-Noise Ratio (SNR) in decibels.

    Returns:
        ndarray: The signal with pink noise added.
    """

    # Generating white noise as an array of size=signal
    white_noise = np.random.normal(0, 1, len(signal))

    # Transforming white noise into fourier domain
    white_noise_fft = np.fft.fft(white_noise)

    # Creating frequency bins
    freqs = np.fft.fftfreq(len(white_noise))

    # Adjusting amplitude in frequency domain for pink noise
    white_noise_fft[freqs != 0] /= np.abs(freqs[freqs != 0])**0.5 # The ratio between larger freqs and their denominators is larger
    white_noise_fft[freqs == 0] = 0

    # Converting back to the time domain and taking the real components
    pink_noise = np.fft.ifft(white_noise_fft).real

    # Adjusting amplitude in time domain for desired SNR
    pink_noise = adjust_SNR(signal, pink_noise)
    return signal + pink_noise

                      #################

def add_white_noise(signal): #snr=[10,17]
    """
    Add white noise to the input signal.

    Parameters:
        signal (ndarray): The input signal to which white noise will be added.
        snr (float): The desired Signal-to-Noise Ratio.

    Returns:
        ndarray: The signal with white noise added.
    """
    # Generating white noise array from Gaussian distribution
    white_noise = np.random.normal(0, 1, len(signal))

    # Adjusting amplitude in time domain for desired SNR
    white_noise = adjust_SNR(signal,white_noise)

    return signal + white_noise

____________
##### Defining the function for adding noise.

In [ ]:
def apply_noise(audio_path, output_path, filename, sr=44100):
    """
    Apply white and pink noises to an audio file.

    Parameters:
        audio_path (str): Path to the input audio file.
        output_path (str): Directory to save the augmented audio files.
        filename (str): Name of the input audio file.
        sr (int, optional): Sampling rate of the audio file. Default is 44100.

    Returns:
        tuple: Original signal, sampling rate,
               filenames of augmented white noise audio, and
               filenames of augmented pink noise audio.
    """

    # Load the file
    signal, sr = librosa.load(audio_path, sr=sr, mono=True)

    # Add Noises
    sig_white_noise = add_white_noise(signal)
    sig_pink_noise = add_pink_noise(signal)


    # Setting up the the output paths for the .wav files
    # White_noise
    white_filename = f"white_{filename}"
    white_path = os.path.join(output_directory, white_filename)

    # Pink_noise
    pink_filename = f"pink_{filename}"
    pink_path = os.path.join(output_directory, pink_filename)

    # Save the augmented audio
    sf.write(white_path, sig_white_noise, sr)
    sf.write(pink_path, sig_pink_noise, sr)

    return signal, sr, white_filename, pink_filename

________________
### Saving the Files

#### Saving Audio Augmented with Noise

In [ ]:
audio_files_directory = '/content/drive/My Drive/UCL/ML_Y3/Project/audio'
output_directory = '/content/drive/My Drive/UCL/ML_Y3/Project/audio/synthetic_audio_V3' # Changing to V3
audio_files = os.listdir(audio_files_directory)

meta = getMeta()

for index, row in meta[['filename', 'target']].iterrows():
    filename = row['filename']
    label = row['target']
    # Identifying a specific sound file which will be augmented
    audio_path = os.path.join(audio_files_directory, filename)

    # Applying the augmentaitons
    signal, sr, white_filename, pink_filename = apply_noise(audio_path, output_directory, filename)

    if index % 50 == 49: # Printing to check progress
      print(f"\n{index}: Processed and saved: {white_filename} with label: {label}")
      print(f"{index}: Processed and saved: {pink_filename} with label: {label}")


49: Processed and saved: white_1-16568-A-3.wav with label: 3
49: Processed and saved: pink_1-16568-A-3.wav with label: 3

99: Processed and saved: white_1-208757-A-2.wav with label: 2
99: Processed and saved: pink_1-208757-A-2.wav with label: 2

149: Processed and saved: white_1-28135-B-11.wav with label: 11
149: Processed and saved: pink_1-28135-B-11.wav with label: 11

199: Processed and saved: white_1-40154-A-46.wav with label: 46
199: Processed and saved: pink_1-40154-A-46.wav with label: 46

249: Processed and saved: white_1-51170-A-18.wav with label: 18
249: Processed and saved: pink_1-51170-A-18.wav with label: 18

299: Processed and saved: white_1-60676-A-34.wav with label: 34
299: Processed and saved: pink_1-60676-A-34.wav with label: 34

349: Processed and saved: white_1-77160-A-3.wav with label: 3
349: Processed and saved: pink_1-77160-A-3.wav with label: 3

399: Processed and saved: white_1-99958-A-31.wav with label: 31
399: Processed and saved: pink_1-99958-A-31.wav with 

##### Debugging the data

In [ ]:
audio_files_directory = '/content/drive/My Drive/UCL/ML_Y3/Project/audio'
output_directory = '/content/drive/My Drive/UCL/ML_Y3/Project/audio/synthetic_audio_V3'

# Checking the number of files that were saved in the output directory
num_files = len([f for f in os.listdir(output_directory) if os.path.isfile(os.path.join(output_directory, f))])
print(f'There were {num_files} number of .wav files saved in the folder')

There were 4000 number of .wav files saved in the folder


There seems to be no missing files. Because we are not applying any complex operations to the sound files. We are only adding noise.

#### Creating dfs

In order to loop through files in my directory.

In [ ]:
# Generating a df for white noised data
meta_white = getMeta()
meta_white['filename'] = "white_" + meta_white['filename']

# Displaying df
meta_white

,filename,fold,target,category,esc10,src_file,take
0,white_1-100032-A-0.wav,1,0,dog,True,100032,A
1,white_1-100038-A-14.wav,1,14,chirping_birds,False,100038,A
2,white_1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A
3,white_1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B
4,white_1-101296-A-19.wav,1,19,thunderstorm,False,101296,A
...,...,...,...,...,...,...,...
1995,white_5-263831-B-6.wav,5,6,hen,False,263831,B
1996,white_5-263902-A-36.wav,5,36,vacuum_cleaner,False,263902,A
1997,white_5-51149-A-25.wav,5,25,footsteps,False,51149,A
1998,white_5-61635-A-8.wav,5,8,sheep,False,61635,A


In [ ]:
# Generating a df for pink noised data
meta_pink = getMeta()
meta_pink['filename'] = "pink_" + meta_pink['filename']

# Displaying df
meta_pink

,filename,fold,target,category,esc10,src_file,take
0,pink_1-100032-A-0.wav,1,0,dog,True,100032,A
1,pink_1-100038-A-14.wav,1,14,chirping_birds,False,100038,A
2,pink_1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A
3,pink_1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B
4,pink_1-101296-A-19.wav,1,19,thunderstorm,False,101296,A
...,...,...,...,...,...,...,...
1995,pink_5-263831-B-6.wav,5,6,hen,False,263831,B
1996,pink_5-263902-A-36.wav,5,36,vacuum_cleaner,False,263902,A
1997,pink_5-51149-A-25.wav,5,25,footsteps,False,51149,A
1998,pink_5-61635-A-8.wav,5,8,sheep,False,61635,A


##### Saving White Noised Files

In [ ]:
data_white = np.zeros((2000,128,500)) # we have 1995 white spectrograms

# sr is the same for the augmented images
sr = 44100

for i in tqdm(range(0, 2000)):
    spec = normalize(getSignalSpectSynth(meta_white, i, ver = 3)[1]) # Looping through only the white files
    spec = fix_audio_length(spec, sr) # Keeping the audio length 500
    data_white[i] = spec


# Saving it on google drive so we don't have to re-run the code
save_file_path = '/content/drive/My Drive/UCL/ML_Y3/Project/images/Data_Arrays/white_spectogram_data_array.npy'
np.save(save_file_path, data_white)

100%|██████████| 2000/2000 [12:08<00:00,  2.75it/s]


##### Saving Pink Noised Files

In [ ]:
data_pink = np.zeros((2000,128,500)) # we have 1995 white spectrograms

# sr is the same for the augmented images
sr = 44100

for i in tqdm(range(0, 2000)):
    spec = normalize(getSignalSpectSynth(meta_pink, i, ver = 3)[1]) # Looping through only the white files
    spec = fix_audio_length(spec, sr) # Keeping the audio length constant
    data_pink[i] = spec


# Saving it on google drive so we don't have to re-run the code
save_file_path = '/content/drive/My Drive/UCL/ML_Y3/Project/images/Data_Arrays/pink_spectogram_data_array.npy'
np.save(save_file_path, data_pink)

100%|██████████| 2000/2000 [13:48<00:00,  2.41it/s]


# Generating Audio for Task 2

In [ ]:
save_file_path = '/content/drive/My Drive/UCL/ML_Y3/Project/images/Data_Arrays/waveform_signal.npy'
signal_array = np.load(save_file_path) # Already normalized between -1 and 1

In [ ]:
save_file_path = '/content/drive/My Drive/UCL/ML_Y3/Project/images/Data_Arrays/waveform_signal.npy'
signal_array = np.load(save_file_path) # Already normalized between -1 and 1

def remove_silence(audio, threshold=0.02):
    # Identify indices where audio is above the threshold
    for i in range(2000):
        non_silent_indices = np.where(np.abs(audio[i,:]) > threshold)

        # Select only those parts of the audio where it is not silent
        audio_non_silent = audio[i,non_silent_indices]

    return audio_non_silent

signal_array_non_silent = remove_silence(signal_array, threshold=0.02)

In [ ]:
signal_array_non_silent.shape

(1, 23755)